# Xgboost workshop (R version)
by Qi XU & Maxime KINET

The goal of this notebook is to present the use of XGBoost in a practical case and some of the functionalities which makes it so useful. XGBoost is an implementation of a Gradient Boosted decision tree algorithm and is frequently used in winning solutions of data science competitions. XGBoost is an open source library, implemented in C++ with wrappers available in Python and R (among others). We have created two versions of this notebook, one in R and one in Python.

The dataset we have chosen to illustrate our purposes comes from the recent B2C competition - Predict conversion rates on breakdown services. The goal was to predict the likelihood of residential customers to subscribe to a maintenance contract. 

More information on this dataset is available [here](https://datascience-challenge.engie.com/#/challenge/28733).


## Before we begin...

If you want to be able to run the notebook, you need to have XGBoost installed on your machine. Installation information is available on the [doc page of XGBoost](https://xgboost.readthedocs.io/en/latest/build.html).

#### R users

If you are a R user, XGBoost is now available as a CRAN package hence a simple `install.packages('xgboost')` should get you up and running.

Aside from XGBoost, the following libraries are expected to be installed :

1. ggplot2 (for graphics)
2. Ckmeans.1d.dp (for variable importance plots)


#### Python users

Python users can use pip to install xgboost *provided* that they are not using windows. On windows, installation can be more complicated and we refer to the documentation for complete instructions.

Aside from xgboost, we expect users to have the following libraries installed :
1. pandas (for data frame manipulation)
2. numpy (for scientific computing)
3. matplotlib (for visualization)


## A few words on XGBoost

A lot has been written on XGBoost since its wide adoption by the datascience community. If you want to learn more about it, here are some links where you can start :

1. [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/)
2. [XGBoost Github repository](https://github.com/dmlc/xgboost)
3. [A presentation by Tianqi Chen](https://www.youtube.com/watch?v=zyjYlU7in3I)
4. [A reference paper](http://dmlc.cs.washington.edu/data/pdf/XGBoostArxiv.pdf)


## Import XGBoost and other packages 

In [ ]:
%matplotlib inline
import xgboost as xgb
import pandas as pd

## Load train and test data sets

We assume that the files `train.CSV` and `test.CSV` are located in the same directory as the notebook. If it is not the case, you can replace the variable `workdir` by the full path to your working directory.

In [ ]:
workdir="./"
# Read the csv file and store the data in a pandas dataframe
train=pd.read_csv(workdir+"train.CSV")
print('The Training set contains %d observations and %d variables' % train.shape)

In [ ]:
test =pd.read_csv(workdir+"test.CSV")
print('The Test set contains %d observations and %d variables' % test.shape)

In [ ]:
# Isolate target variable
target = train["souscrit"]
# Define the target variable for the test set as NaN.
test["souscrit"] = None
# Bind Training and Test set. We do this to make sure that every operation is performed identically 
# on the training and on the test set
db = pd.concat([train,test],axis=0)
print('The Data set contains %d observations and %d variables' % db.shape)

## Pre-processing

Before firing XGBoost, we need to prepare the dataset. For this simple notebook, we will perform two tasks :
1. Handle Missing values
2. Convert all non-numerical variables to numerical.

### Missing Values

The data set contains many missing values on several variables. Let us count how many values are missing for each.

In [ ]:
# this will print a summary of Missing Values count for each variable
db.isnull().sum()

There are different strategies to impute missing value and usually we would carefully look at the data and decide what is the best strategy for variables with missing values. However, for the sake of this exercice, we will use one of the nice feature of XGBoost which allow us to do ***nothing*** about missing value and leave them as they are.

There are potentially 41 predictors, but we will not use all of them to facilitate the treatment in this tutorial. We will exclude for instance all the categorical variable and keep the numerical variables and the dates. 

In [ ]:
# Names of the numerical variables
num_var_name = ['AGE_TR_PRED_FMT','CAP_EC', 'CAR_CHAR', 'CEL_FMT', 'CRC', 'DARTY', 'FACT_LIGN_FMT',
                'MAILING', 'MA_RLV_FMT','NB_COHABITANT', 'PAP', 'PARTENAIRES', 'PAYEUR_DIVERG', 'PREA', 'PUIS_SOUS', 
                'REVENU_IRIS', 'TV', 'ass_fact', 'nb_cont_COUR_ENT', 'nb_cont_MAIL_ENT', 'nb_cont_RECLAMATION',
                'nb_cont_RECLAMATION_6M', 'nb_cont_TEL_ENT', 'nb_contact_entrant']

As mentionned earlier, the dates need to be converted to numerical variables to be usable by XGBoost. For each date we can extract :
* the month (number from 1 to 12)
* the day of the week (number from 1 to 7)
* the day of the year (number from 1 to 365)
* the year

For the variable `DT_DEBT_ASSR` we will extract the first three features, while for `DT_ANC_CLI` and `DT_EMMENAG` we will keep only the year which, intuitively is the most meaningful.

In [ ]:
# Transform of timestamps
def time_attr_frame(timestamp):
    month    = timestamp.month
    week_day = timestamp.weekday()
    yday    = timestamp.timetuple().tm_yday # day of year (leak feature)
    return month,week_day,yday

def extract_year():
    pass

# Extract month, day of week and day of year from DT_DEBT_ASSR
db["DT_DEBT_ASSR"] = pd.to_datetime(db["DT_DEBT_ASSR"],format='%Y-%m-%d')
db["DT_DEBT_ASSR_month"],db["DT_DEBT_ASSR_week_day"],db["DT_DEBT_ASSR_yday"]=zip(*db["DT_DEBT_ASSR"].map(time_attr_frame))

# Tranform DATE_ANC_CLI and DATE_EMMENAG to years
db["DATE_ANC_CLI_year"]=db["DATE_ANC_CLI"].map(lambda x: int(x[0:4]))
db["DATE_EMMENAG_year"]=db["DATE_EMMENAG"].map(lambda x: int(x[0][0:4]))

Now we can construct the final data set from the numerical variables and the modified timestamps.

In [ ]:
db_final = db[num_var_name + ["DT_DEBT_ASSR_yday","DT_DEBT_ASSR_month","DT_DEBT_ASSR_week_day","DATE_ANC_CLI_year",\
                              "DATE_EMMENAG_year"]]

And finally, we can split into training and test set and start building XGBoost models.

In [ ]:
# seperate train & test
train = db_final.iloc[0:127472,:]
test  = db_final.iloc[127472:,:]

In [ ]:
# define parameters we are going to use
params_1 = {"nthread":3,
          "objective": "binary:logistic",
          "eval_metric": "auc",
          "eta": 0.5,
          "max_depth": 1}
num_trees = 100 

### First Model

The algorithm will be trained on the training set. To evaluate the performance of the algorithm accurately, we have to use data that the algorithm has never seen, otherwise the model will be overfitted. 

We will randomly select a subset from the training set and use it as a validation set. That is, we will predict the target on the validation set, and compare it to the true class to obtain a rough estimate of what the auc will be on the test set.

In [ ]:
# validation set consist of 20000 randomly selected clients
xvalid=train.sample(n=20000,random_state=200)
idx=xvalid.index
xtrain=train.drop(idx)
target_train=target.drop(idx)
target_valid=target[idx]

The next important step is to convert the training and validation set to a DMatrix format (which is a xgboost-specific format). We do that in the following piece of code.

In [ ]:
dtrain    = xgb.DMatrix(xtrain, target_train, missing = None)
dvalid    = xgb.DMatrix(xvalid, target_valid, missing = None)

Side note : `None` is the default for missing parameter in python, but you can change it if you use another convention in your dataset. 
For instance, you can choose that all values equal to `-99999` correspond to a missing value and xgboost will treat it as missing.

Now let us train the model. In Xgboost, we can pass as argument a *watchlist* which is a list of dataset for which we wish to compute the performance metric regularly. In the watch list, we put our validation set and our training set.

In [ ]:
watchlist = [(dvalid,"valid"),(dtrain,"train")]
xgb_simple = xgb.train(params_1,dtrain,num_trees,watchlist,verbose_eval=10)

Great! It works and we can even expect a pretty decent score (auc ~ 0.76). Now let us see how we can improve the algorithm.

### Second model

First, we should improve our validation methodolgy. Watchlist are useful for fast evaluation, but they can be inacurate. We will make a proper cross-validation instead.

Second there is the possibility in XGBoost to tell it to stop when the evaluation metric (on the validation data) does not improve for a certain number of iterations. This is very useful to evaluate the number of iterations that are needed to optimize performance. We will tell xgboost to step if the auc does not increase for 10 iterations

In [ ]:
# rebuild the DMatrix with full training set.
dtrain    = xgb.DMatrix(train, target, missing = None)
# Set the maximum number of trees to a high value
num_trees = 1000
# Launch 5-fold cross- validation.
xgb_cv = xgb.cv(params_1,dtrain,num_trees,nfold=5,early_stopping_rounds=10,verbose_eval=25,seed=519)
# find optimal number of iterations
print('Highest AUC is %f obtained at iteration %d' % (xgb_cv['test-auc-mean'].max(),xgb_cv['test-auc-mean'].idxmax()))

### Tuning

Now we can play with some parameters of XGBoost. We can modify parameters to improve the accuracy of the booster:
1. Reducing the learning rate : eta = 0.1
2. Increasing the depth of the trees : we allow trees to grow a bit deeper (up to depth =5).

Next, since the trees we grow are more complex, we might want to tune other parameters to avoid overfitting
1. min_child_weight defines the minimum (weighted) number of observations in a child to split a node.
2. subsample : to introduce randomization, we can use only a subset of the training set to grow the trees. We set it to .9 (i.e. each tree will be trained on 90% of the data).

With these adjusted parameters we can retrain the model.

In [ ]:
# define New set of parameters
params_2 = {"nthread":3,
          "objective": "binary:logistic",
          "eval_metric": "auc",
          "eta": 0.1,
          "max_depth": 4,
          "min_child_weight": 20,
          "subsample": 1.0
                              }

# Launch 5-fold cross- validation.
xgb_cv = xgb.cv(params_2,
                dtrain,
                num_trees,
                nfold=5,
                early_stopping_rounds=10,
                verbose_eval=25,
                seed=519)
# find optimal number of iterations
print('Highest AUC is %f obtained at iteration %d' % (xgb_cv['test-auc-mean'].max(),xgb_cv['test-auc-mean'].idxmax()))

Fantastic, we have improved our score!

### Variable Importance

Another great feature of XGBoost is its ability to compute an importance score for the predictors. This allow to understand better on what ground the prediction are made. 

In the code snippet below, we first retrain the model on the full training set, with the optimal parameters found above, and then we plot the variables importance in a bar chart.

In [ ]:
# train xgboost on the full training set
xgb_final = xgb.train(params_2,dtrain,160)
# feature importance to see which features are more importants than others
xgb.plot_importance(xgb_final,importance_type="gain",xlabel="gain")

### Test set prediction 

The last think that remain to do is to make a prediction on the test set, and to prepare the submission file. 

In [ ]:
dtest = xgb.DMatrix(test)
pred = xgb_final.predict(dtest)

In [ ]:
submission = pd.read_csv(workdir+"sample_submission.CSV")
submission["predictions"]=pred
submission.to_csv("myfirst_submission.CSV",index=False)

## Congratulations !!

You have completed your first xgboost workshop!

Few notes :

- XGBoost is also available in scikit-learn, which is definitely worth exploring
- Things we have not covered : sparse matrix, other metrics, customised objectives,...
- There are most certainly some XGBoost tricks that we don't know about!